# DeepDRR nifti_msg_to_volume Test

In [33]:
import os
from pathlib import Path


patient_data_dir = Path(os.environ.get("PATIENT_DATA_DIR", ""))
print(patient_data_dir)

patientCaseID = "case-100114"
boneType = "THIN_BONE_TORSO"
volumePrefix = "nifti"
volumeName = "THIN_BONE_TORSO_STANDARD_TORSO_Thorax_?????_??"
volumeSuffix = ".nii.gz"
volumeId = f'{volumePrefix}/{boneType}/{patientCaseID}/{volumeName}{volumeSuffix}'

niftiParamsPath = Path(volumeId)
if not niftiParamsPath.expanduser().is_absolute():
    niftiWildcardsPath = patient_data_dir / niftiParamsPath
else:
    niftiWildcardsPath = niftiParamsPath
print(f"niftiWildcardsPath [{type(niftiWildcardsPath)}]: {niftiWildcardsPath}")

niftiCaseDir = niftiWildcardsPath.parent
niftiVolumeName = niftiWildcardsPath.name
print(f"niftiCaseDir [{type(niftiCaseDir)}]: {niftiCaseDir}")
print(f"niftiVolumeName [{type(niftiVolumeName)}]: {niftiVolumeName}")

niftiPaths = sorted(niftiCaseDir.glob(niftiVolumeName)) + [niftiWildcardsPath]
niftiPath = str(niftiPaths[0])
print(f"niftiPath [{type(niftiPath)}]: {niftiPath}")

/nfs/centipede/liam/OneDrive/NMDID-ARCADE
niftiWildcardsPath [<class 'pathlib.PosixPath'>]: /nfs/centipede/liam/OneDrive/NMDID-ARCADE/nifti/THIN_BONE_TORSO/case-100114/THIN_BONE_TORSO_STANDARD_TORSO_Thorax_65008_12.nii.gz
niftiCaseDir [<class 'pathlib.PosixPath'>]: /nfs/centipede/liam/OneDrive/NMDID-ARCADE/nifti/THIN_BONE_TORSO/case-100114
niftiVolumeName [<class 'str'>]: THIN_BONE_TORSO_STANDARD_TORSO_Thorax_65008_12.nii.gz
niftiPath [<class 'str'>]: /nfs/centipede/liam/OneDrive/NMDID-ARCADE/nifti/THIN_BONE_TORSO/case-100114/THIN_BONE_TORSO_STANDARD_TORSO_Thorax_65008_12.nii.gz


# Patient Case Search

In [2]:
import os
from pathlib import Path


BASE_DIRS = {
    'root1': Path(os.environ.get("PATIENT_DATA_DIR", "")).expanduser(),
    'root2': Path.home() / "datasets" / "DeepDRR_DATA"
}

CONSTANTS = {
    'PatientCaseIDPrefix': "case-*",
    'TorsoPrefix': "THIN_BONE_TORSO",
    'PatientMeshLoaderPrefix': "TotalSegmentator_mesh",
    'PatientMeshLoaderMeshname': "body",
    'PatientMeshLoaderSuffix': ".stl",
    'NiftiDRRVolumePrefix': "nifti",
    'NiftiDRRVolumeSuffix': ".nii.gz",
    'PatientAnnoLoaderPrefix': "2023-pelvis-annotations",
    'PatientAnnoLoaderSuffix': ".mrk.json"
}


print(BASE_DIRS['root1'])
print(BASE_DIRS['root2'])


def compute_paths(base_dir, prefixes):
    return {key: base_dir / value / CONSTANTS['TorsoPrefix'] for key, value in prefixes.items()}

prefixes = {
    'mesh_loader': CONSTANTS['PatientMeshLoaderPrefix'],
    'nifti_volume': CONSTANTS['NiftiDRRVolumePrefix'],
    'anno_loader': CONSTANTS['PatientAnnoLoaderPrefix']
}

suffixes = {
    'mesh_loader': CONSTANTS['PatientMeshLoaderMeshname'] + CONSTANTS['PatientMeshLoaderSuffix'],
    'nifti_volume': CONSTANTS['NiftiDRRVolumeSuffix'],
    'anno_loader': CONSTANTS['PatientAnnoLoaderSuffix']
}

paths = compute_paths(BASE_DIRS['root1'], prefixes)


def search_common_cases(paths, prefix):
    common_cases = None
    for path in paths.values():
        case_set = {case_dir.name for case_dir in path.glob(prefix) if case_dir.is_dir()}
        common_cases = case_set if common_cases is None else common_cases & case_set
    return sorted(common_cases)


def search_files_for_cases(common_cases, paths):
    case_files = {}
    for case_id in common_cases:
        case_files[case_id] = {
            file_type: sorted((paths[file_type] / case_id).rglob(f"*{suffixes[file_type]}"))
            for file_type in ['mesh_loader', 'nifti_volume', 'anno_loader']
            if (paths[file_type] / case_id).exists() and (paths[file_type] / case_id).is_dir()
        }
    return case_files


PatientCaseID = search_common_cases(paths, CONSTANTS['PatientCaseIDPrefix'])
case_files = search_files_for_cases(PatientCaseID, paths)


case_id = 3
# print(case_files)
# print(case_files[PatientCaseID[case_id]])
print(f'mesh = {case_files[PatientCaseID[case_id]]["mesh_loader"]}')
print(f'nifti = {case_files[PatientCaseID[case_id]]["nifti_volume"]}')
# print(f'anno = {case_files[PatientCaseID[case_id]]["anno_loader"]}')


/nfs/centipede/liam/OneDrive/NMDID-ARCADE
/home/virtualpelvislab/datasets/DeepDRR_DATA
mesh = [PosixPath('/nfs/centipede/liam/OneDrive/NMDID-ARCADE/TotalSegmentator_mesh/THIN_BONE_TORSO/case-100155/body.stl')]
nifti = [PosixPath('/nfs/centipede/liam/OneDrive/NMDID-ARCADE/nifti/THIN_BONE_TORSO/case-100155/THIN_BONE_TORSO_STANDARD_TORSO_Thorax_90156_12.nii.gz'), PosixPath('/nfs/centipede/liam/OneDrive/NMDID-ARCADE/nifti/THIN_BONE_TORSO/case-100155/nnunet_tmp_827CQIOP/s01_0000.nii.gz'), PosixPath('/nfs/centipede/liam/OneDrive/NMDID-ARCADE/nifti/THIN_BONE_TORSO/case-100155/nnunet_tmp_SVH5O8N3/s01_0000.nii.gz')]


# .bashrc Path Test

In [1]:
from pathlib import Path
import os

from deepdrrzmq.utils.zmq_util import zmq_no_linger_context
from deepdrrzmq.utils.typer_util import unwrap_typer_param
from deepdrrzmq.utils.server_util import make_response

default_data_dir = Path("/mnt/d/jhonedrive/Johns Hopkins/Benjamin D. Killeen - NMDID-ARCADE/")  # TODO: remove
patient_data_dir = Path(os.environ.get("PATIENT_DATA_DIR", default_data_dir))
patient_data_dir2 = Path(os.getenv("PATIENT_DATA_DIR", default_data_dir))

print(patient_data_dir)
print(patient_data_dir2)

/nfs/centipede/liam/OneDrive/NMDID-ARCADE
/nfs/centipede/liam/OneDrive/NMDID-ARCADE


kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/virtualpelvislab


# loggercontroller

In [ ]:
    # async def ui_update_loop(self):
    #     last_heard_time = 0
    #     while True:
    #         latest_msgs = {}

    #         try:
    #             print(f"enter try")
    #             topic, data = await self.ZMQAMH_instance.sub_socket.recv_multipart()
    #             latest_msgs[topic] = data
    #         except zmq.ZMQError:
    #             print(f"enter except")
    #             pass
    #         print(f"exit try except")
            
    #         for topic, data in latest_msgs.items():
    #             if topic == b"/loggerd/status/":
    #                 last_heard_time = time.time()
    #                 self.record_button.disabled = False
    #                 self.stop_button.disabled = False
    #                 with messages.LoggerStatus.from_bytes(data) as msg:
    #                     self.session_id.value = msg.sessionId if msg.recording else "no session"
    #                     self.record_status.value = str(msg.recording)
    #                     self.record_button.button_style = 'danger' if msg.recording else ''

    #         if time.time() - last_heard_time > 3:
    #             self.session_id.value = "Disconnected"
    #             self.record_status.value = "Disconnected"
    #             self.record_button.disabled = True
    #             self.stop_button.disabled = True

    #         await asyncio.sleep(0.1)

In [ ]:
# record_button = widgets.Button(description='New Session', disabled=True, button_style='', icon='circle')
# stop_button = widgets.Button(description='Stop', disabled=True, button_style='', icon='square')

# session_id = widgets.Text(value='None', placeholder='', description='Session ID:', disabled=True)
# record_status = widgets.Text(value='None', placeholder='', description='Recording:', disabled=True)

# async def f():
#     last_heard_time = 0

#     while True:
#         latest_msgs = {}

#         try:
#             # for i in range(1000):
#             #     topic, data = ZMQAMH.sub_socket.recv_multipart(flags=zmq.NOBLOCK)
#             #     latest_msgs[topic] = data
#             topic, data = await ZMQAMH.sub_socket.recv_multipart()
#             latest_msgs[topic] = data
#         except zmq.ZMQError:
#             pass

#         for topic, data in latest_msgs.items():
#             if topic == b"/loggerd/status/":
#                 last_heard_time = time.time()
#                 record_button.disabled = False
#                 stop_button.disabled = False
#                 with messages.LoggerStatus.from_bytes(data) as msg:
#                     session_id.value = msg.sessionId if msg.recording else "no session"
#                     record_status.value = str(msg.recording)
#                     record_button.button_style = 'danger' if msg.recording else ''

#         if time.time() - last_heard_time > 3:
#             session_id.value = "Disconnected"
#             record_status.value = "Disconnected"
#             record_button.disabled = True
#             stop_button.disabled = True

#         await asyncio.sleep(0.1)

# try:
#     ui_update_task.cancel()
# except:
#     pass
# ui_update_task = asyncio.create_task(f())

# async def on_record_button_clicked(b):
#     await ZMQAMH.pub_socket.send_multipart([b"/loggerd/start/", b""])
#     print(f"Record button clicked, b = {b}")
    
# async def on_stop_button_clicked(b):
#     await ZMQAMH.pub_socket.send_multipart([b"/loggerd/stop/", b""])
#     print(f"Stop button clicked, b = {b}")

# colortoggle = widgets.ToggleButtons(
#     options=[('Opaque', 0), ('Transparent' ,1)],
#     description='Patient Material:',
#     disabled=False,
#     button_style='', # 'success', 'info', 'warning', 'danger' or ''
# )

# viewindicator = widgets.ToggleButtons(
#     options=[('On',True), ('Off',False)],
#     description='View Indicator ON/OFF:',
#     disabled=False,
#     button_style='', # 'success', 'info', 'warning', 'danger' or ''
# )

# selfviewindicator = widgets.ToggleButtons(
#     options=[('On',True), ('Off',False)],
#     description='View Indicator Self-Select ON/OFF:',
#     disabled=False,
#     button_style='', # 'success', 'info', 'warning', 'danger' or ''
# )

# flippatient = widgets.ToggleButtons(
#     options=[('Original',True), ('Flipped',False)],
#     description='Flip Patient:',
#     disabled=False,
#     button_style='', # 'success', 'info', 'warning', 'danger' or ''
# )

# kwireindicator = widgets.ToggleButtons(
#     options=[('On',True), ('Off',False)],
#     description='Kwire Error Indicator ON/OFF:',
#     disabled=False,
#     button_style='', # 'success', 'info', 'warning', 'danger' or ''
# )

# kwireerrorselection = widgets.ToggleButtons(
#     options=[('On',True), ('Off',False)],
#     description='Kwire Error Indicator Self-Selection',
#     disabled=False,
#     button_style='', # 'success', 'info', 'warning', 'danger' or ''
# )


# corridorselection= widgets.Select(
#     options=[('Left S1', 0), ('Left Ramus', 1), ('L Ramus Short', 2),('L Teardrop', 3),('Right S1', 4), ('Right Ramus', 5), ('Right Ramus Short', 6),('Right Teardrop', 7),('S 2', 8),('S 3', 9),('None',100)],
#     value=2,
#     description='Corridor:',
# )


# update_setting = widgets.Button(
#     description='Update Setting',
#     disabled=False,
#     button_style='', # 'success', 'info', 'warning', 'danger' or ''
#     tooltip='Click me',
#     icon='check' # (FontAwesome names without the `fa-` prefix)
# )

# viewindicatorselect = widgets.SelectMultiple(
#     options=['Anteroposterior_View', 'Lateral_View', 'Inlet_View', 'Outlet_View', 'Obturator_Oblique_Left/Iliac_Oblique_Right', 'Obturator_Oblique_Right/Iliac_Oblique_Left', 'Obturator_Oblique_Inlet_Left/Iliac_Oblique_Inlet_Right','Obturator_Oblique_Outlet_Left/Iliac_Oblique_Outlet_Right','Obturator_Oblique_Inlet_Right/Iliac_Oblique_Inlet_Left','Obturator_Oblique_Outlet_Right/Iliac_Oblique_Outlet_Left', 'Teardrop_Left_View','Teardrop_Right_View','None'],
#     value=['Lateral_View'],
#     #rows=10,
#     description='View Indicator Selection:',
#     disabled=False
# )

# patient_model_layout = widgets.VBox([colortoggle,flippatient])
# view_indicator_layout = widgets.VBox([viewindicator,selfviewindicator,viewindicatorselect])
# surgical_instrument_layout = widgets.VBox([kwireindicator,kwireerrorselection,corridorselection])
# display(widgets.HBox([patient_model_layout,view_indicator_layout ,surgical_instrument_layout,update_setting]))

# async def updatesetting_clicked(b):
#     response_topic = "/mp/setting/webgui/SettingManager/"
#     msg = messages.SycnedSetting.new_message()
#     msg.timestamp = msg.timestamp = float(np.float64(time.time()).item())
#     msg.clientId = 'webgui'
#     uiControlVar = msg.setting.init("uiControl")
#     uiControlVar.patientMaterial = colortoggle.value
#     uiControlVar.flippatient = flippatient.value #need add      
#     uiControlVar.carmIndicator = viewindicator.value
#     uiControlVar.viewIndicatorselfselect = selfviewindicator.value #need add
#     uiControlVar.annotationError = viewindicatorselect.value#for different view selection
#     uiControlVar.corridorIndicator =kwireindicator.value
#     uiControlVar.webcorridorerrorselect = kwireerrorselection.value
#     uiControlVar.webcorridorselection = corridorselection.value
#     await ZMQAMH.pub_socket.send_multipart([response_topic.encode(), msg.to_bytes()])
#     print(f"Update Setting button clicked, b = {b}")

# update_setting.on_click(lambda b: asyncio.create_task(updatesetting_clicked(b)))
# record_button.on_click(lambda b: asyncio.create_task(on_record_button_clicked(b)))
# record_button.on_click(lambda b: asyncio.create_task(updatesetting_clicked(b)))
# stop_button.on_click(lambda b: asyncio.create_task(on_stop_button_clicked(b)))



# display(widgets.HBox([record_button, stop_button, session_id, record_status]))